https://cookbook.openai.com/examples/gpt4o/introduction_to_gpt4o

In [ ]:
import sys
sys.path.append("./api")

import generate_openai
import generate_claude
import generate_gemini

In [ ]:
MAX_NEW_TOKEN = 512

In [ ]:
from datasets import load_dataset
dataset = load_dataset("elyza/ELYZA-tasks-100", split="test")

In [ ]:
dataset

In [ ]:
dataset["input"][0]

In [ ]:
for d in dataset:
    print(d)

In [ ]:
openai_result = generate_openai.chat_complettion(model="gpt-4o", user_prompt=dataset["input"][0])
openai_result

In [ ]:
def get_message_from_complettion_json(response_json):
    return response_json["choices"][0]["message"]["content"]
openai_result = get_message_from_complettion_json(openai_result)

In [ ]:
model, tokenizer = generate_local_llm.load_model("elyza/ELYZA-japanese-Llama-2-7b-instruct")

In [ ]:
text = dataset["input"][0]
prompt = generate_local_llm.build_llama_prompt(text, tokenizer)

In [ ]:
local_llm_result = generate_local_llm.generate(model, tokenizer, prompt=prompt, max_new_tokens=MAX_NEW_TOKEN)
local_llm_result

In [ ]:
prompt = generate_claude.format_claude_prompt(dataset["input"][0])
claude_result = generate_claude.chat_complettion(model="anthropic.claude-v2:1", prompt=prompt, max_tokens_to_sample=MAX_NEW_TOKEN)
claude_result = claude_result.get('completion')

## GPT-4Judgeを入れる
判定基準：https://huggingface.co/datasets/elyza/ELYZA-tasks-100

In [ ]:
gpt4_judge_prompt_ans = """
[指示]
公平な判断者として行動し、以下に表示されるユーザーの質問に対するAIアシスタントの応答の品質を評価してください。
あなたの評価は正確さと有用性を考慮すべきです。AIアシスタントの返答の言語は、ユーザーが使用している言語と一致しているべきで、そうでない場合は減点されるべきです。
参照答えと判定基準とアシスタントの答えが与えられます。あなたの評価は、アシスタントの答えと参照答えを比較することから始めてください。
比較後、あなたの評価は、アシスタントの答えを判定基準に従って評価してください。判定基準で判定できない場合、タスク評価基準に従って評価してください。
ミスを特定し、訂正してください。できるだけ客観的であること。
説明を提供した後、このフォーマットに厳密に従って1から10までのスケールで応答を評価する必要があります：\"[[評価]]\"、例えば：\"評価：[[5]]\"。

[質問]
{question}

[参考回答の開始]
{ref_answer_1}
[参考回答の終了]

[判定基準の開始]
{eval_aspect}
[判定基準の終了]

[タスク評価基準の開始]
{tasks_aspect}
[タスク評価基準の終了]

[アシスタントの回答の開始]
{answer}
[アシスタントの回答の終了]
"""

In [ ]:
elyza_tasks_aspect = """
[評価基準]
点数についての基本的な評価基準は、以下のようになります。

[基本的な評価基準]
ベースとなる得点:
- 1点: 誤っている
 - 指示に従えていない場合
 - 選択肢を選ぶ問題で、誤った選択肢を選んでいる場合
 - 筆記問題で全体的に事実と異なる内容を述べている場合
 - 部分的に事実と異なる内容を述べている場合は、減点項目にあるように部分的な誤りとして-1点の減点のみになります。
- 2点: 誤っているが、方向性は合っている
 - 基本的に2点をつけることは少なく、3点から1点減点した場合に2点になる場合が多いです
 - 複数の問題や指示が与えられ、ほとんど間違っているがわずかに（1/3以下）正解している場合
 - 1つの入力に4問の指示が与えられ、そのうちの1つのみが正解している場合など
 - 後述の、過度に安全性を気にしていて答えられていない場合
- 3点: 部分的に誤っている, 部分的に合っている
 - 複雑な指示が与えられ、そのうちの一部（半分以上）のみに従えている場合
 - フレンドリーなスタイルで要約してという指示に対し、要約はできているがフレンドリーな文体ではない場合など
 - 複数の問題が与えられ、そのうちの一部（半分以上）のみが正解している場合
 - 1つの入力に4問の指示が与えられ、そのうちの2~3つ正解している場合など
- 4点: 合っている
 - 問いに対して正解している場合
 - 正解しているが、答えのみのぶっきらぼうな回答であるが場合や、当てずっぽうで選択肢を当てている場合
- 5点: 役に立つ
 - 問いや指示に対して正しい回答をし、その上でユーザーが何に困っているかを想像し、役に立とうとしている場合
 - 例: 生徒の要約を改善するタスクで、ただ改善した要約を提示するだけでなく、どのような点を改善すればいいのかを説明している場合
- 問いに対して正解していて、その上で理由や説明を提示していて役に立つ場合
 - ブレインストームや執筆系のタスクにおいて、想像力豊かな回答をしている場合

 [基本的な減点項目]
ベースとなる得点から、以下のような要素を考慮して、得点を調整します。

- 不自然な日本語: -1点
 - 一見して意味がわからない不自然な日本語、同じ文章の繰り返し、唐突に現れる英語
- 部分的な誤り: -1点
 - 筆記問題で部分的に事実と異なる内容を述べている場合
  - 全体的に事実と異なる内容を述べている場合、前述のベースとなる得点が1点になります（-4点扱い）
  - 一見して事実か判断がつかない場合は、Googleなどで検索してチェックをしてください。
  - ただし全てのファクトチェックを行うと作業が終わらないので、特に怪しい箇所などをランダムにピックアップして調べるのでもOKです。
- 過度な安全性: 2点にする
 - 「倫理的に答えられません」というような回答
"""

In [ ]:
openai_result

In [ ]:
local_llm_result

In [ ]:
claude_result

In [ ]:
dataset["output"][0]

In [ ]:
dataset["eval_aspect"][0]

In [ ]:
ind = 0
openai_eval = gpt4_judge_prompt_ans.format(question=dataset["input"][ind], ref_answer_1=dataset["output"][ind], eval_aspect=dataset["eval_aspect"][ind], tasks_aspect=elyza_tasks_aspect, answer=openai_result)
local_llm_eval = gpt4_judge_prompt_ans.format(question=dataset["input"][ind], ref_answer_1=dataset["output"][ind], eval_aspect=dataset["eval_aspect"][ind], tasks_aspect=elyza_tasks_aspect, answer=local_llm_result)
claude_eval = gpt4_judge_prompt_ans.format(question=dataset["input"][ind], ref_answer_1=dataset["output"][ind], eval_aspect=dataset["eval_aspect"][ind], tasks_aspect=elyza_tasks_aspect, answer=claude_result)

In [ ]:
openai_eval

In [ ]:
local_llm_eval

In [ ]:
claude_eval

In [ ]:
openai_eval_result = generate_openai.chat_complettion(model="gpt-4o", user_prompt=openai_eval)
local_llm_eval_result = generate_openai.chat_complettion(model="gpt-4o", user_prompt=local_llm_eval)
claude_eval_result = generate_openai.chat_complettion(model="gpt-4o", user_prompt=claude_eval)

In [ ]:
openai_eval_result = get_message_from_complettion_json(openai_eval_result)
local_llm_eval_result = get_message_from_complettion_json(local_llm_eval_result)
claude_eval_result = get_message_from_complettion_json(claude_eval_result)

In [ ]:
import pandas as pd
df = pd.read_csv("ElyzaTasks100/judge_gpt-4-turbo/gpt-4-turbo.csv")

In [ ]:
import re
df["score"] = df["ELYZA-tasks-100評価結果"].apply(lambda x: re.search("\[\[(.*)\]\]", x).group(0))
df["score"] = df["score"].str.replace("[", "").str.replace("]", "")
df["score"].astype(int).mean()